<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Теоретическое-обоснование-преобразований" data-toc-modified-id="Теоретическое-обоснование-преобразований-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Теоретическое обоснование преобразований</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span><ul class="toc-item"><li><span><a href="#Посчитаем-матрицу-XP" data-toc-modified-id="Посчитаем-матрицу-XP-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Посчитаем матрицу XP</a></span></li><li><span><a href="#Посчитаем-качество-модели-на-исходных-данных" data-toc-modified-id="Посчитаем-качество-модели-на-исходных-данных-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Посчитаем качество модели на исходных данных</a></span><ul class="toc-item"><li><span><a href="#Выделим-признаки-для-обучения" data-toc-modified-id="Выделим-признаки-для-обучения-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Выделим признаки для обучения</a></span></li><li><span><a href="#Выделим-целевой-признак" data-toc-modified-id="Выделим-целевой-признак-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Выделим целевой признак</a></span></li><li><span><a href="#Обучим-модель,-и-оценим-ее-качество" data-toc-modified-id="Обучим-модель,-и-оценим-ее-качество-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Обучим модель, и оценим ее качество</a></span></li></ul></li></ul></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Посчитаем-качество-модели-на-закодированных-данных" data-toc-modified-id="Посчитаем-качество-модели-на-закодированных-данных-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Посчитаем качество модели на закодированных данных</a></span><ul class="toc-item"><li><span><a href="#Выделим-признаки-для-обучения" data-toc-modified-id="Выделим-признаки-для-обучения-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Выделим признаки для обучения</a></span></li><li><span><a href="#Обучим-модель-и-оценим-ее-качество" data-toc-modified-id="Обучим-модель-и-оценим-ее-качество-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>Обучим модель и оценим ее качество</a></span></li></ul></li></ul></li></ul></div>

# Защита персональных данных клиентов

# Постановка задачи

Нужно защитить данные клиентов страховой компании. Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

# Данные

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
STATE = 12345

In [3]:
df = pd.read_csv('/datasets/insurance.csv')
df.info()
df.shape
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


## Теоретическое обоснование преобразований

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

**Гипотеза:** если признаки умножить на обратимую матрицу, качество линейной регрессии не изменится 

**Обоснование:** поскольку умножение на обратимую матрицу является линейным преобразованием, не меняющим линейные зависимости между признаками и целевым признкаом, то модель линейной регрессии учится на преобразованных данных с такой же точночностью, что и до преобразования.

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Требуется доказать, что  если $Х = XP$, где P - квадратная обратимая матрица, то предсказания $a = Xw$ не изменятся. Заменим в формуле вектора весов $Х$  на $XP$:
$$
w = ((XP)^T XP)^{-1} (XP)^T y
$$
Используя свойство $(AB)^T = B^T A^T$, запишем
$w = (P^T Х^T XP)^{-1} P^T X^T y$.  
Учитывая что в скобках - 3 матрицы квадратные обратимые:$P^T, Х^TX и P$ используем свойство:
$(A B)^{-1} = B^{-1} A^{-1}$  
$w = (P)^{-1} (Х^TX)^{-1} (P^T)^{-1} P^T X^T y$.
Так как $(P^T)^{-1} P^T = 1$  
$w = (P)^{-1} (Х^TX)^{-1} X^T y$  
Подставим в формулу расчета предсказаний  
$a = XPw = X P (P)^{-1} (Х^T X)^{-1} X^T y$ 
так как 
$P (P)^{-1} = 1$  
$a = X (Х^T X)^{-1} X^T = Xw$


## Алгоритм преобразования

### Посчитаем матрицу XP

Преобразуем датафрейм в матрицу, исключим последний столбец с целевым признаком

In [4]:
X = df.iloc[:,:-1].values
X.shape

(5000, 4)

Создадим случайную квадратную матрицу и проверим ее на обратимость. Размер ее 4 * 4, чтобы можно было умножить на первноначальную матрицу 5000*4

In [5]:
P = np.random.normal(size = (4,4)) 
display(P)
display(P.shape)
P_inv = np.linalg.inv(P)
P_inv

array([[-0.3840361 ,  0.20803752,  0.3807728 ,  0.78359209],
       [-2.33722356, -0.31765328,  0.98454262,  0.10862054],
       [-1.8174773 ,  2.31777848,  1.68044956, -0.96094644],
       [ 0.53090468, -1.63951729, -0.03425082, -1.24976788]])

(4, 4)

array([[ 0.95322375, -0.59554607,  0.14183179,  0.43684696],
       [-0.38275276, -0.15799621,  0.1714369 , -0.38553187],
       [ 2.04549731, -0.44535585,  0.41141867,  0.92745974],
       [ 0.85099063, -0.03351572, -0.17592549, -0.13422972]])

In [6]:
B = X @ P
B.shape

(5000, 4)

### Посчитаем качество модели на исходных данных

#### Выделим признаки для обучения

In [7]:
features = df.drop(columns='Страховые выплаты', axis =1)
features

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


#### Выделим целевой признак

In [8]:
target = df['Страховые выплаты']
target

0       0
1       1
2       0
3       0
4       0
       ..
4995    0
4996    0
4997    0
4998    0
4999    0
Name: Страховые выплаты, Length: 5000, dtype: int64

#### Обучим модель, и оценим ее качество

In [9]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
score = r2_score(target, predictions)
score

0.42494550286668

## Проверка алгоритма

### Посчитаем качество модели на закодированных данных

In [10]:
df_coded = pd.DataFrame(B, index=df.index, columns=df.columns[:-1])
df_coded

,Пол,Возраст,Зарплата,Члены семьи
0,-90242.553144,114947.357429,83391.010980,-47658.956323
1,-69171.118600,88059.330737,63902.338016,-36512.218070
2,-38234.802684,48664.136171,35317.992511,-20176.725314
3,-75836.823099,96641.412934,70095.353574,-40071.685191
4,-47501.983703,60485.332119,43887.681500,-25076.877204
...,...,...,...,...
4995,-64948.319892,82732.518471,60019.548009,-34305.246188
4996,-95314.744970,121439.152713,88088.997179,-50351.150300
4997,-61658.162972,78563.058430,56986.862458,-32576.411554
4998,-59481.717797,75779.657466,54972.638593,-31423.524757


#### Выделим признаки для обучения

In [11]:
features_coded = df_coded

#### Обучим модель и оценим ее качество

In [12]:
model_coded = LinearRegression()
model_coded.fit(features_coded, target)
predictions_coded = model_coded.predict(features_coded)
score = r2_score(target, predictions_coded)
score

0.42494550286669

# Вывод
Результат R2 на трансформированных данных -  0.4249455028666803 такой же как на исходных.